<a href="https://colab.research.google.com/github/AnatoliyFominV/Kazan_Express/blob/main/Fast_text_and_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kazan Express
Соревнование по классификации товаров по их описанию

## Exploratory data analis

#### Импорт пакетов и данных

In [ ]:
!pip install catboost;
!pip install igraph;
import igraph
import catboost
import pandas as pd
import numpy as np
from google.colab import drive
from google.colab import output
from catboost import CatBoostClassifier, Pool, metrics, cv
from sklearn.metrics import accuracy_score
output.enable_custom_widget_manager()
drive.mount('/content/drive')

file_name = 'train.parquet'
df = pd.read_parquet(f'/content/drive/My Drive/Colab Notebooks/Data/{file_name}')

file_name = "categories_tree.csv"
category_tree = pd.read_csv(f'/content/drive/My Drive/Colab Notebooks/Data/{file_name}')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.1 MB 7.4 MB/s 
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df.describe()

,id,rating,feedback_quantity,category_id
count,2.834520e+05,283452.000000,283452.00000,283452.000000
mean,9.401805e+05,1.814641,4.34656,12244.567295
std,4.191303e+05,2.309150,34.67245,2543.823093
min,9.500000e+01,0.000000,-1.00000,2598.000000
25%,6.063808e+05,0.000000,0.00000,12049.000000
50%,9.963495e+05,0.000000,0.00000,12730.000000
75%,1.298164e+06,4.871795,2.00000,13408.000000
max,1.655762e+06,5.000000,6102.00000,14559.000000


In [ ]:
df.sample(10)

,id,title,short_description,name_value_characteristics,rating,feedback_quantity,category_id
32163,681969,"Очищающая пенка для умывания, 150мл",Очищающая пенка с матирующим эффектом,None,5.0000,6,13269
155647,1310527,Память USB 2.0/ Флеш-накопитель USB/ USB-Флешк...,"USB накопитель Perfeo с колпачком, модель C09 ...",Объем памяти:8Gb|16Gb|32Gb|64Gb,0.0000,0,13786
72401,934300,Поднос ажурный с ручками пластиковый 29х40 см,Поднос ажурный с ручками пластиковый 29х40 см ...,None,0.0000,0,13474
63655,1469408,Ремень текстильный тактический нейлоновый тяну...,None,None,0.0000,0,12618
178384,1284230,Сумка мужская через плечо из экокожи,None,None,0.0000,0,12556
227197,1482631,Системный разъем microUSB №63,None,None,0.0000,0,13066
270331,767466,Верхнее покрытие E.Co Nails Dalmatian Top Coat...,None,None,0.0000,0,13021
262433,1210873,Повязка для девочек / Бантик для волос,Ручная работа Цена за 1 шт.,None,5.0000,2,13274
15488,590304,Карточки для фотосессии и записей малыша новор...,Карточки для фотосессии и записей малыша новор...,None,0.0000,0,12545
248069,744355,Почтовые открытки новогодние,None,None,4.9375,16,13407


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 283452 entries, 0 to 283451
Data columns (total 7 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   id                          283452 non-null  int64  
 1   title                       283452 non-null  object 
 2   short_description           133130 non-null  object 
 3   name_value_characteristics  50360 non-null   object 
 4   rating                      283452 non-null  float64
 5   feedback_quantity           283452 non-null  int64  
 6   category_id                 283452 non-null  int64  
dtypes: float64(1), int64(3), object(3)
memory usage: 15.1+ MB


In [ ]:
df.short_description.fillna("", inplace=True);
df.name_value_characteristics.fillna("", inplace=True);

### Устраняем дисбаланс классов

In [ ]:
def take_n_samles(x):
    n = 1000
    n_object_in_class = g.size()[x.category_id].iloc[0]
    
    if n > n_object_in_class:
        return x.sample(n_object_in_class).reset_index(drop=True)
    elif n <= n_object_in_class:
        return x.sample(n).reset_index(drop=True)

df_class_balanced = df.copy()  # Скопируем дф
g = df_class_balanced.groupby("category_id")  # Сгруппируем по категориям
df_balanced = g.apply(take_n_samles)  # Возьмём по 60 товаров в каждой категории, или меньше
df_balanced = df_balanced.reset_index(drop=True)  # Удалим индексы (они дублируют столбец категорий) 

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df_balanced, test_size=0.5, stratify=df_balanced.category_id)
y_train = train["category_id"]
x_train = train.drop(columns=["id","category_id"])  # Удаляем id рейтинг и число отзывов они не влияют на категорию

test = df.drop(train.index)
y_test = test["category_id"]
x_test = test.drop(columns=["id","category_id"])

In [ ]:
# Число классов в трейне
category_in_train = len(train.groupby("category_id")["id"].count())
category_in_test = len(test.groupby("category_id")["id"].count())
print(f"Число категорий в обучающей {category_in_train}, тестовой: {category_in_test}")

Число категорий в обучающей 1231, тестовой: 1215


## Fastex

#### Utils for F1 hierarhical

In [ ]:
from __future__ import annotations
from typing import Union, Tuple
import os
from pathlib import Path
import pickle
import tqdm
import pandas as pd
import numpy as np
from collections import Iterable


class CategoryTree(object):

    def __init__(self) -> None:
        self.tree = dict()
        self.proba_cache = dict()
        self.curr_dir = Path(".").parent
        return

    def __getitem__(self, node_id: int) -> None:
        return self.tree[node_id]

    def __len__(self) -> int:
        return len(self.tree)

    def keys(self) -> Iterable:
        return self.tree.keys()

    def items(self) -> Iterable:
        return self.tree.items()

    def save_tree(self, fname: str, directory: str = '') -> None:
        path = os.path.join(self.curr_dir, directory, fname)
        with open(path, 'wb') as f:
            pickle.dump(self.tree, f)
        return

    def load_tree(self, fname: str, directory: str = '') -> None:
        path = os.path.join(self.curr_dir, directory, fname)
        with open(path, 'rb') as f:
            self.tree = pickle.load(f)

    def add_nodes_from_df(self, data: pd.DataFrame, parent_id_col: str, title_col: str) -> None:
        for key, row in data.iterrows():
            self.tree[key] = {'parent_id': row[parent_id_col],
                              'children_ids': set(),
                              'title': row[title_col],
                              'ancestral_titles': set(),
                              'embedding': np.array([]),
                              'weights': np.array([]),
                              'intercept': 0.,
                              'good_count': 0,
                              'good_ids': []
                              }
        self.update_ancestors()
        return

    def add_goods_from_df(self, data: pd.DataFrame, category_id_col: str, good_id_col: str) -> None:
        for key, row in data.iterrows():
            self.add_good(row[category_id_col], row[good_id_col])
        return

    def add_good(self, cat_id: int, good_id: int) -> None:
        curr_id = cat_id
        while True:
            self.tree[curr_id]['good_ids'].append(good_id)
            self.tree[curr_id]['good_count'] += 1
            next_id = self[curr_id]['parent_id']
            if next_id == 0:
                break
            self.tree[curr_id]['ancestral_titles'].add(self[next_id]['title'])
            self.tree[curr_id]['ancestral_titles'].update(self[next_id]['ancestral_titles'])
            self.tree[next_id]['children_ids'].add(curr_id)
            curr_id = next_id
        return

    def update_ancestors(self) -> None:
        for i in range(3):
            for curr_id in self.keys():
                next_id = self.tree[curr_id]['parent_id']
                if next_id == 0 or next_id not in self.keys():
                    continue
                self.tree[curr_id]['ancestral_titles'].add(self[next_id]['title'])
                self.tree[curr_id]['ancestral_titles'].update(self[next_id]['ancestral_titles'])
        return

    def update_embeddings(self, embeddings_dict: dict) -> None:
        dim = embeddings_dict[next(iter(embeddings_dict))].shape[0]
        for _, node in self.items():
            if node['good_count'] > 0:
                node['embedding'] = np.zeros((dim,), dtype='float32')
                for good_id in node['good_ids']:
                    node['embedding'] += embeddings_dict[good_id] / node['good_count']
        for _, node in self.items():
            node['embedding'] = node['embedding'] / (np.linalg.norm(node['embedding']) + 0.001)
        return

    def mix_in_description_embs(self, encoder: callable, weight: int = 4) -> None:
        id_list = list(self.keys())
        path_titles = []
        for node_id in id_list:
            path_title = [self[node_id]['title']] + list(self[node_id]['ancestral_titles']).copy()
            if 'Все категории' in path_title:
                path_title.remove('Все категории')
            path_titles.append(' '.join(path_title))
        title_embeddings = encoder(path_titles)
        for node_id, embedding in zip(id_list, title_embeddings):
            if self.tree[node_id]['embedding'].shape[0] == 0:
                self.tree[node_id]['embedding'] = embedding
            else:
                self.tree[node_id]['embedding'] = (self.tree[node_id]['embedding'] * (
                    self.tree[node_id]['good_count']) ** 0.5
                                                   + embedding * weight) / (
                                                          (self.tree[node_id]['good_count']) ** 0.5 + weight)
        return

    def get_siblings_ids(self, node_id: int) -> [set | None]:
        if (node_id not in self.tree) or (self[node_id]['good_count'] == 0):
            return None
        parent = self[node_id]['parent_id']
        if parent == 0:
            return None
        childs = self[parent]['children_ids'].copy()
        if not childs:
            return None
        childs.remove(node_id)
        return childs

    def choose_leaf(self, classifier: callable, good_embedding: np.array = None, document: str = None) -> int:
        current_id = 1
        next_id = 1
        while next_id:
            current_id = next_id
            next_id, _ = self.choose_child(current_id,
                                           classifier=classifier,
                                           good_embedding=good_embedding,
                                           document=document)
        self.proba_cache = dict()
        return current_id

    def choose_leaf_proba(self, classifier: callable,
                          good_embedding: np.array = None,
                          document: str = None) -> Tuple[int, float]:
        next_id = 1
        current_id = 1
        leaf_proba = 1.
        while next_id:
            current_id = next_id
            next_id, proba = self.choose_child(current_id,
                                               classifier=classifier,
                                               good_embedding=good_embedding,
                                               document=document)
            leaf_proba *= proba
        self.proba_cache = dict()
        return current_id, leaf_proba

    def choose_leaf_proba_upwards(self, leaf_ids_probas: List[Tuple[int, float]],
                                  classifier: callable,
                                  good_embedding: np.array, proba_power: float = 1.,
                                  sib_proba_power: float = 0.) -> Tuple[int, float]:
        max_proba = 0.
        best_leaf = 1
        for leaf_id, leaf_proba in leaf_ids_probas:
            next_node = self.tree[leaf_id]['parent_id']
            proba = leaf_proba
            while next_node > 1:
                proba *= self.eval_node_proba(node_id=next_node,
                                              classifier=classifier,
                                              good_embedding=good_embedding) ** proba_power
                siblings = self.get_siblings_ids(next_node)
                sibl_proba = sib_proba_power
                for sibling in siblings:
                    sibl_proba += self.eval_node_proba(node_id=sibling,
                                                       classifier=classifier,
                                                       good_embedding=good_embedding)
                proba /= (sibl_proba / (len(siblings) + 0.5) ** 0.3)
                next_node = self.tree[next_node]['parent_id']
            if proba > max_proba:
                max_proba = proba
                best_leaf = leaf_id
        return best_leaf, max_proba

    def choose_child(self, node_id: int, classifier: callable,
                     good_embedding: np.array = None,
                     document: str = None) -> Union[None, tuple]:

        childs = self[node_id]['children_ids']
        if len(childs) == 0:
            return None, 1.
        if len(childs) == 1:
            return list(childs)[0], 1.0
        max_proba = 0.
        best_child = None
        for child in childs:
            proba = self.eval_node_full_proba(child,
                                              classifier=classifier,
                                              good_embedding=good_embedding,
                                              document=document)
            if max_proba < proba:
                max_proba = proba
                best_child = child
        return best_child, max_proba

    def eval_node_full_proba(self, node_id: int,
                             good_embedding: np.array = None,
                             document: str = None,
                             classifier: callable = None) -> float:
        """Probability of node membership for certain commodity with considering child nodes probabilities"""
        self_proba = self.eval_node_proba(node_id,
                                          classifier=classifier,
                                          good_embedding=good_embedding,
                                          document=document)

        childs = self[node_id]['children_ids']
        if len(childs) == 0:
            return self_proba
        max_proba = 0.
        sum_proba = 0.
        for child in childs:
            proba = self.eval_node_proba(child,
                                         classifier=classifier,
                                         good_embedding=good_embedding,
                                         document=document)
            sum_proba += proba ** 2
            if max_proba < proba:
                max_proba = proba
        return (max_proba * 0.7 + sum_proba ** 0.5 * 0.3) * self_proba

    def eval_node_proba(self, node_id: int,
                        classifier: callable,
                        good_embedding: np.array = None,
                        document: str = None) -> float:
        """Probability of node membership for certain commodity without considering child nodes probabilities"""
        if node_id in self.proba_cache:
            return self.proba_cache[node_id]
        elif document:
            proba = classifier.predict_local_proba_by_doc(node_id, document)
            print(proba)
        elif len(self[node_id]['weights']) == 0:
            return 1e-8
        else:
            proba = classifier.predict_local_proba(self[node_id]['weights'],
                                                   self[node_id]['intercept'],
                                                   self[node_id]['embedding'],
                                                   good_embedding)
        self.proba_cache[node_id] = proba
        return proba

    def get_id_path(self, node_id: int) -> list:
        path = []
        current_id = node_id
        while current_id > 1:
            path.append(current_id)
            current_id = self[current_id]['parent_id']
        return path

    def get_id_path_set(self, node_id: int) -> set:
        path = set()
        current_id = node_id
        while current_id > 1:
            path.add(current_id)
            current_id = self[current_id]['parent_id']
        return path

    def hF1_score(self, true_leafs: list, pred_leafs: list) -> Union[None, float]:
        if len(true_leafs) != len(pred_leafs):
            return None
        sum_Ti = 0
        sum_Pi = 0
        sum_Ti_Pi = 0
        for true, pred in zip(true_leafs, pred_leafs):
            pr_rec_tuple = self.get_good_pr_rec(true, pred)
            sum_Ti += pr_rec_tuple[0]
            sum_Pi += pr_rec_tuple[1]
            sum_Ti_Pi += pr_rec_tuple[2]

        hP = sum_Ti_Pi / sum_Pi
        hR = sum_Ti_Pi / sum_Ti
        return 2 * hP * hR / (hP + hR)

    def hF1_score_01(self, true_leafs, pred_leafs) -> Union[None, float]:
        if len(true_leafs) != len(pred_leafs):
            return None
        sum_Ti = 0
        sum_Pi = 0
        sum_Ti_Pi = 0
        for true, pred in zip(true_leafs, pred_leafs):
            pr_rec_tuple = self.get_good_pr_rec(true, pred)
            sum_Ti += pr_rec_tuple[0] + 2
            sum_Pi += pr_rec_tuple[1] + 2
            sum_Ti_Pi += pr_rec_tuple[2] + 2

        hP = sum_Ti_Pi / sum_Pi
        hR = sum_Ti_Pi / sum_Ti
        return 2 * hP * hR / (hP + hR)

    def get_good_pr_rec(self, good_category_id: int, predicted_leaf_id: int) -> tuple:
        Ti_set = self.get_id_path_set(good_category_id)
        Pi_set = self.get_id_path_set(predicted_leaf_id)
        inter_set = Ti_set.intersection(Pi_set)
        return len(Ti_set), len(Pi_set), len(inter_set)

    def fit_local_weights(self, classifier: object, embeddings_dict: dict, C: int = 1,
                          reg_count_power: float = 0.4, verbose=False) -> None:
        """Fitting of all local node classifiers, saving weights and intercept of logistic regression in tree"""
        for key in tqdm.tqdm(list(self.keys()), total=len(self)):
            siblings = self.get_siblings_ids(key)
            if siblings:
                siblings_goods = []
                for sibling in siblings:
                    siblings_goods += self[sibling]['good_ids']

                dim = embeddings_dict[next(iter(embeddings_dict))].shape[0] + 1
                if self.tree[key]['weights'].shape[0] == dim:
                    warm_coefs = self.tree[key]['weights']
                else:
                    warm_coefs = np.zeros(dim)
                self.tree[key]['weights'], \
                self.tree[key]['intercept'] = classifier.fit_node_weights(self[key]['good_ids'],
                                                                          siblings_goods,
                                                                          self[key]['embedding'],
                                                                          embeddings_dict,
                                                                          C_in=C,
                                                                          reg_count_power=reg_count_power,
                                                                          warm_coefs=warm_coefs,
                                                                          warm_intercept=self.tree[key]['intercept'],
                                                                          verbose=verbose)
        return

#### Model

In [ ]:
!pip install fasttext
import os
from pathlib import Path
import fasttext
import numpy as np 
import pandas as pd
import csv
from gensim.utils import simple_preprocess
from google.colab import drive
drive.mount('/content/drive')

file_name = 'train.parquet'
df = pd.read_parquet(f'/content/drive/My Drive/Colab Notebooks/Data/{file_name}')

file_name = "categories_tree.csv"
category_tree = pd.read_csv(f'/content/drive/My Drive/Colab Notebooks/Data/{file_name}')


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = df.fillna(value="")

In [ ]:
data_full = df[['title', 'short_description', 'category_id']].copy()
data_full["Document_new"] = [name + " " + desc + " __label__" + str(label) for name, desc, label in zip(data_full["title"],	data_full['short_description'],	data_full['category_id'])]
data_full['Document_new'] = data_full['Document_new'].apply(simple_preprocess)
data_full['Document_new'] = data_full['Document_new'].apply(lambda x: " ".join(x[:50]))
data_train = data_full.iloc[:int(249969 * 0.8)].copy()
data_test = data_full.iloc[int(249969 * 0.8):].copy()

In [ ]:
# Write to file for start Fastext
with open("train_baseline.csv","w") as file:
  for des, label in zip(data_train['Document_new'], data_train['category_id']):
    print(f"{des}",end=", ", file=file)
    print(f" __label__" + str(label), file=file)

with open("test_baseline.csv","w") as file:
  for des, label in zip(data_test['Document_new'], data_test['category_id']):
    print(f"{des}",end=", ", file=file)
    print(f" __label__" + str(label), file=file)

In [ ]:
from os.path import exists 

file_name = "fast_text_model_baseline.bin"
path_model = f'/content/drive/My Drive/Colab Notebooks/Data/{file_name}'

if exists(path_model):
  model = fasttext.load_model(path_model)

else:
  model = fasttext.train_supervised(input="train_baseline.csv",
                          lr = 0.3,
                          dim=40,
                          ws=5,
                          wordNgrams=3,
                          minCount=10)
  model.save_model(path_model)

In [ ]:
def make_prediction(corpus):
  ans = []
  res = model.predict(corpus)
  for text in res[0]:
    ans.append(int(text[0][9:]))
  return ans
make_prediction(["рифленный нож слайсер для фигурной нарезки см","рифленный нож слайсер для фигурной нарезки см"])

[14030, 14030]

## BERT

In [ ]:
!pip install transformers
import pandas as pd
import numpy as np
import networkx as nx

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

from transformers import BertModel, BertTokenizer
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

import matplotlib.pyplot as plt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 7.2 MB/s 
     |████████████████████████████████| 596 kB 56.4 MB/s 
     |████████████████████████████████| 101 kB 10.8 MB/s 
     |████████████████████████████████| 6.6 MB 40.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


#### DataSet / Bert

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn


class KEDataset(Dataset):
    def __init__(self, df, tokenizer, max_length=512, is_test=False, target_names=None):
        # target_names - список наименований категорий в one-hot представлении целевой переменной. Формат элементов - str.
        self.df = df
        self.target_names = target_names
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.is_test = is_test

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        
        text  = row["text"]
        if not self.is_test: 
            label = row[self.target_names]
            true_class = row['category_id']

        tokenized_text = self.tokenizer(text, max_length=self.max_length, padding='max_length', truncation=True)

        res = {
            "input_ids": torch.Tensor(tokenized_text["input_ids"]).long(),
            "token_type_ids": torch.Tensor(tokenized_text["token_type_ids"]).long(),
            "attention_mask": torch.Tensor(tokenized_text["attention_mask"]).long(),
            }
        if not self.is_test: res["label"] = torch.Tensor(label).float()
        if not self.is_test: res["true_class"] = true_class
        
        return res
    

class BertClasifyByCLS(nn.Module):
    def __init__(self, bert, emd_size=264, num_classes=2):
        super(BertClasifyByCLS, self).__init__()
        self.bert = bert
        self.num_classes = num_classes
        self.drop_out = nn.Dropout(0.1)
        self.clsf = nn.Linear(emd_size, self.num_classes)

    def forward(self, input_ids_text,
                      token_type,
                      attention_mask_text):
        bert_text_out = self.bert(input_ids_text,
                                  token_type,
                                  attention_mask_text)[0][:,0]
        clsf_input = self.drop_out(bert_text_out)
        return self.clsf(clsf_input)

#### Utils

In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import random, os

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from sklearn.metrics import f1_score

from tqdm import tqdm


def seed_everything(seed: int):   
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

def get_path(G, pred):
    #по предсказанной категории возвращает всю ветку до корня
    # G - граф со структурой категорий
    # pred - предсказанная категория, название категории в формате int
    # возвращает всю ветку категорий от предсказанной категории до корня
    node = pred
    path = [node]
    while list(G.predecessors(node)):
        path.append(list(G.predecessors(node))[0])
        node = list(G.predecessors(node))[0]
    return set(path)

def get_hF(G, y_true, y_pred):
    # G - граф со структурой категорий
    # y_true - истинные метки классов,  названия категории в формате int
    # y_pred - предсказанные метки классов,  названия категории в формате int
    # возвращает рассчитанное значение иерархического f1.
    assert len(y_true) == len(y_pred)
    P_T = 0
    P = 0
    T = 0
    for i in range(len(y_true)):
        P_i = get_path(G, y_pred[i]) 
        T_i = get_path(G, y_true[i])
        P_T += len(P_i.intersection(T_i))
        P += len(P_i)
        T += len(T_i)
    hP = P_T / P
    hR = P_T / T
    hF = 2 * hP * hR / (hP + hR)
    return hF

def train_epoch(model, loss_function, optimizer, train_loader, val_loader,
                num_epoch=3, scheduler=None, device='cuda', 
                needed_features=['input_ids', 'token_type_ids', 'attention_mask']):
    model.train()
    running_loss = 0.0
    num_batches = 0
    step = 0
    for batch in tqdm(train_loader):
        label = batch["label"].to(device)

        seed_everything(42)
        model_out = model(*[batch[fname].to(device) for fname in needed_features])

        batch_loss = loss_function(model_out, label)
        batch_loss.backward()

        torch.nn.utils.clip_grad_norm_(
            model.parameters(), 1.0
        )

        optimizer.step()
        optimizer.zero_grad()

        if scheduler:
            scheduler.step()

        running_loss += batch_loss.detach().cpu().numpy()
        num_batches += 1
    epoch_loss = running_loss / num_batches
    print("train loss: ", epoch_loss)


def val_epoch(model, loss_function, val_loader, device='cpu', target_names = None,
              needed_features=['input_ids', 'token_type_ids', 'attention_mask']):
    # target_names - список наименований категорий в one-hot представлении целевой переменной. Формат элементов - str.
    model.eval()
    running_loss = 0.0
    num_batches = 0
    model_prob = None
    true_label = None

    for batch in val_loader:
        label = batch["label"].to(device)
        true_class = batch["true_class"]
        model_out = model(*[batch[fname].to(device) for fname in needed_features])
        
        batch_prob = torch.softmax(model_out, dim=1).detach().cpu().numpy()
        batch_loss = loss_function(model_out, label)

        if model_prob is None:
            model_prob = batch_prob
            true_label = label.detach().cpu().numpy()
            true_class_model = true_class.detach().cpu().numpy()
        else:
            model_prob = np.vstack((model_prob, batch_prob))
            true_label = np.vstack((true_label, label.detach().cpu().numpy()))
            true_class_model = np.hstack((true_class_model, true_class.detach().cpu().numpy()))

        running_loss += batch_loss.detach().cpu().numpy()
        num_batches += 1
        
    epoch_loss = running_loss/num_batches
    print("val loss: ", epoch_loss)

    pred_idx = np.argmax(model_prob, axis=1)
    pred_label = np.array([int(target_names[i]) for i in pred_idx])  
    epoch_f1 = f1_score(true_class_model, pred_label, average='micro')
    
    return epoch_f1
    

def infer(model, val_loader, device='cpu',
          needed_features=['input_ids', 'token_type_ids', 'attention_mask']):
    model.eval()
    running_loss = 0.0
    num_batches = 0
    model_prob = None
    true_label = None
    for batch in tqdm(val_loader):
        model_out = model(*[batch[fname].to(device) for fname in needed_features])
        batch_prob = torch.softmax(model_out, dim=1).detach().cpu().numpy()
        
        if model_prob is None:
            model_prob = batch_prob
        else:
            model_prob = np.vstack((model_prob, batch_prob))

    return model_prob


def infer_single(model, val_loader, device='cpu',
                 needed_features=['input_ids', 'token_type_ids', 'attention_mask']):
    model.eval()
    running_loss = 0.0
    num_batches = 0
    model_prob = None
    true_label = None
    for batch in tqdm(val_loader):
        model_out = model(*[batch[fname].to(device) for fname in needed_features])["logits"]
        batch_prob = torch.softmax(model_out, dim=1).detach().cpu().numpy()
        
        if model_prob is None:
            model_prob = batch_prob
        else:
            model_prob = np.vstack((model_prob, batch_prob))

    return model_prob


def train_model(model, loss_function, optimizer, train_loader, val_loader, target_names = None,
                num_epoch=3, scheduler=None, device='cpu', name_to_save='model.pt',
                best_val_score=0.0, needed_features=['input_ids', 'token_type_ids', 'attention_mask']):
    # target_names - список наименований категорий в one-hot представлении целевой переменной. Формат элементов - str.
    for epoch in range(num_epoch):
        train_epoch(model, loss_function, optimizer, train_loader, val_loader, 
                    num_epoch, scheduler, device, needed_features)
        
        val_score = val_epoch(model, loss_function, val_loader, device, target_names,
                    needed_features)
        print(f"val f1_score after {epoch+1} epoch: {val_score}")

        if val_score > best_val_score:
            best_val_score = val_score
            print("new best score! Saving model...")
            torch.save(model, name_to_save)
        print("*"*100)

#### Code

## CatBoost

In [ ]:
text_features = [0, 1, 2]
train_dataset = Pool(data=x_train,
                     label=y_train,            
                     text_features=text_features)

eval_dataset = Pool(data=x_test,
                    label=y_test,
                    text_features=text_features)

In [ ]:
# Initialize CatBoostClassifier
model_1000 = CatBoostClassifier(iterations=50,
                           learning_rate=0.1,
                           depth=2,
                           loss_function='MultiClass',
                           task_type="GPU",
                           custom_loss=["Accuracy"],
                           save_snapshot=True,
                           snapshot_file="snapshot_1000.bkp",
                           snapshot_interval=1,
                           random_seed=43               
                           
                           )  

In [ ]:
# Fit model

history = model_1000.fit(train_dataset,
                         eval_set=eval_dataset,
          logging_level='Verbose',
          plot=False
          )

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [ ]:
model_1000.save_model(fname='/content/drive/My Drive/Colab Notebooks/Data/Flat_model_100',
                 format="cbm",)

### Validation

In [ ]:
model = CatBoostClassifier(iterations=50,
                           learning_rate=0.1,
                           depth=2,
                           loss_function='MultiClass',
                          #  task_type="GPU",
                           custom_loss=["Accuracy"],
                           save_snapshot=True,
                           snapshot_file="snapshot.bkp",
                           snapshot_interval=1,
                           random_seed=43                   
                           
                           )  

model.load_model('/content/drive/My Drive/Colab Notebooks/Data/Flat_model')

In [ ]:
preds_class = model.predict(eval_dataset)

NameError: ignored

In [ ]:
print(f'Simple model validation accuracy: {accuracy_score(y_test, preds_class)}')

NameError: ignored

### F1 Score

In [ ]:
from catboost import CatBoostClassifier, Pool, metrics, cv

model = CatBoostClassifier(iterations=50,
                           learning_rate=0.1,
                           depth=2,
                           loss_function='MultiClass',
                          #  task_type="GPU",
                           custom_loss=["Accuracy"],
                           save_snapshot=True,
                           snapshot_file="snapshot.bkp",
                           snapshot_interval=1,
                           random_seed=43                   
                           
                           )  
model.load_model('/content/drive/My Drive/Colab Notebooks/Data/Flat_model')

In [ ]:
preds_class = model.predict(eval_dataset)

In [ ]:
# f1 calculate
from sklearn.metrics import f1_score

f1_score(y_test, preds_class, average='weighted')

0.3417219401932432